<h1 style='color:orange;'>Import des librairies</h1>

In [1]:
import pandas as pd
import joblib 

In [2]:
# Stockage des couleurs de titre
h1_color= 'orange'
h2_color= '#A1DC12'
h3_color='#4397FF'
h4_color='#57BEF7'

<h1 style='color:orange;'>Fonctions</h1>

<h2 style='color:#A1DC12;'>Remplacement des valeurs manquantes</h2>

In [3]:
def nan_fill(df, reg):
    '''
    Remplace les valeurs manquantes de la colonne 'margin_low' par les valeurs prédites par la régression linéaire
    '''

    # Récupération des valeurs manquantes
    df_nan = df.loc[df['margin_low'].isna()]

    # Variables de prédictions
    if len(df_nan) > 0:
        predict_var = df_nan[['length', 'margin_up', 'height_right', 'height_left', 'diagonal']]

        # Prédiction 
        predict_nan = reg.predict(predict_var)

        # Remplacement des valeurs manquantes
        df.loc[df['margin_low'].isna(), 'margin_low'] = predict_nan

    
    return df

<h1 style='color:orange;'>Import du pipeline</h1>

In [4]:
pipeline = joblib.load('models/pipeline.joblib')
reg_robustscaled = pipeline['nan_predict']
robust_scaler = pipeline['nan_scaler']
clf_standard = pipeline['model']
log_standard_scaler = pipeline['log_scaler']
best_threshold = pipeline['best_threshold']
features = pipeline.get('features', ['diagonal''length','height_left','height_right','margin_low','margin_up'])
impute_features = pipeline.get('impute_features', ['length','margin_up','height_right','height_left'])


<h1 style='color:orange;'>Import et préparation du fichier</h1>

<h2 style='color:#A1DC12;'>Import</h2>

In [5]:
df = pd.read_csv('billets_production.csv', sep = None, engine = 'python' )
df.head()

,diagonal,height_left,height_right,margin_low,margin_up,length,id
0,171.76,104.01,103.54,5.21,3.30,111.42,A_1
1,171.87,104.17,104.13,6.00,3.31,112.09,A_2
2,172.00,104.58,104.29,4.99,3.39,111.57,A_3
3,172.49,104.55,104.34,4.44,3.03,113.20,A_4
4,171.65,103.63,103.56,3.77,3.16,113.33,A_5


<h2 style='color:#A1DC12;'>Vérification des colonnes</h2>

In [6]:
missing_cols = set(impute_features + features) - set(df.columns)
if missing_cols:
    raise ValueError(f"Colonnes manquantes dans le fichier: {missing_cols}")

<h2 style='color:#A1DC12;'>Gestion des valeurs manquantes</h2>

<h3 style='color:#4397FF;'>De 'margin_low'</h3>

In [7]:
# Dataframe ne contenant que les lignes pour lesquelles margin_low est NaN
if df['margin_low'].isna().any():
    df_nan=df.loc[df['margin_low'].isna()]

    # Variables de prédiction
    predict_var=df_nan[['length', 'margin_up', 'height_right', 'height_left']]
    predict_var_scaled = robust_scaler.transform(predict_var)
    predict_var_scaled = pd.DataFrame(predict_var_scaled, columns = predict_var.columns, index = predict_var.index)
    
    # Prédiction pour les valeurs NaN
    predict_nan=reg_robustscaled.predict(predict_var_scaled)
    
    # Remplacement des valeurs NaN
    df.loc[df['margin_low'].isna(), 'margin_low'] = predict_nan

<h3 style='color:#4397FF;'>Recherche d'autre valeurs manquantes</h3>

In [8]:
columns = df.columns
has_nan = False

# Liste des colonnes contenant des valeurs manquantes
nan_col = []

# Nombre de valeurs manquantes par colonnes
for col in columns:
    n_nan = df[col].isna().sum()
    if n_nan > 0:
        has_nan = True
        nan_col.append(col)
        print(f'''
        ❌   {n_nan} valeurs manquantes dans la colonne {col}
        ''')

if not has_nan: 
    print(f'''
    ✅   Aucune valeur manquante détectée
    ''')


    ✅   Aucune valeur manquante détectée
    


<h2 style='color:#A1DC12;'>Gestion des doublons</h2>

In [9]:
df = df.drop_duplicates()

<h1 style='color:orange;'>Préparation des données pour la régression logistique et prédictions</h1>

In [10]:
# Définition de X
X = df[clf_standard.feature_names_in_].copy()

# Application du scaler entrainé
X_scaled = pd.DataFrame(
    log_standard_scaler.transform(X), 
    columns = clf_standard.feature_names_in_, 
    index = X.index
)

# Prédictions
pred = clf_standard.predict(X_scaled)
proba = clf_standard.predict_proba(X_scaled)[:, 1]
pred_opt = (proba >= best_threshold).astype(int)

# Ajouter les prédictions au dataset
df['prediction'] = [True if p == 1 else False for p in pred_opt]
df['probabilities'] = proba

df.head()

,diagonal,height_left,height_right,margin_low,margin_up,length,id,prediction,probabilities
0,171.76,104.01,103.54,5.21,3.30,111.42,A_1,False,0.002306
1,171.87,104.17,104.13,6.00,3.31,112.09,A_2,False,0.000352
2,172.00,104.58,104.29,4.99,3.39,111.57,A_3,False,0.000489
3,172.49,104.55,104.34,4.44,3.03,113.20,A_4,True,0.968568
4,171.65,103.63,103.56,3.77,3.16,113.33,A_5,True,0.999790


In [11]:
# Affichage du nombre de vrais et des faux billets
n_true = df['prediction'].sum()
n_false = df.shape[0] - n_true
n_true_pct = (n_true / df.shape[0] * 100).round(1)
n_false_pct = (n_false / df.shape[0] * 100).round(1)

# Liste des vrais et des faux billets
true_bill = df.loc[df['prediction'] == True, 'id'].tolist()
false_bill = df.loc[df['prediction'] == False, 'id'].tolist()


# Billets incertains
threshold = 10
high_threshold = best_threshold * (1 + (threshold / 100))
low_threshold = best_threshold * (1 - (threshold / 100))
uncertain = (df.loc[
             (df['probabilities'] > low_threshold) &
             (df['probabilities'] < high_threshold), 
             'id'
             ].tolist()
            )
n_uncertain = len(uncertain)
pct_uncertain = round(n_uncertain / df.shape[0] * 100, 1)


print(f'''
Nombre de vrais billets: {n_true} ({n_true_pct}%)
{true_bill}
Nombre de faux billets: {n_false} ({n_false_pct}%)
{false_bill}
Billets à vérifier: {n_uncertain} ({pct_uncertain}%)
{uncertain}
''')

if has_nan:
    print(f''' 
    ⚠️    Attention, présence de valeurs manquantes
    {nan_col}
    ''')


Nombre de vrais billets: 2 (40.0%)
['A_4', 'A_5']
Nombre de faux billets: 3 (60.0%)
['A_1', 'A_2', 'A_3']
Billets à vérifier: 0 (0.0%)
[]

